In [ ]:
import json
import os
from glob import glob
import numpy as np
from typing import List

In [ ]:
import sys
sys.path.insert(0, "./ai")

In [ ]:
from AI import *

In [ ]:
processed_components_folder = 'assets/media/processed_components'

In [ ]:
components_in_pack = ['Object 1', 'Object 3'] # The components you want to train

In [ ]:
all_components = []
all_labels = []
all_sizes = []
for comp in components_in_pack:
    for img_uri in glob(os.path.join(processed_components_folder, comp) + "/*.png"):
        try:
            size = img_uri.split(".png")[-2]
            with open(size + '--size.json', "r") as f:
                all_sizes.append(json.load(f))
                img = imread(img_uri)[:,:,:3] / 255
                all_components.append(img)
                all_labels.append(comp)
        except FileNotFoundError:
            pass



In [ ]:


all_labels_np = np.array(all_labels)
all_components_np = np.array(all_components)
all_sizes_np = np.array(all_sizes)



In [ ]:


print(all_labels_np.shape)
print(all_components_np.shape)
print(all_sizes_np.shape)



In [ ]:


from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder



In [ ]:


label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(all_labels_np)

# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)

import math

training_indices = []
test_indices = []
for obj in range(0, onehot_encoded.shape[1]):
    obj_indices = np.where(onehot_encoded[:,obj] == 1)[0]
    np.random.shuffle(obj_indices)
    training_samples = math.floor(0.7 * len(obj_indices))
    training_indices.extend(obj_indices[:training_samples])
    test_indices.extend(obj_indices[training_samples:])

print(f"{len(training_indices)} training indices")
print(f"{len(test_indices)} test indices")



In [ ]:


X_train_conv = all_components_np[training_indices]
X_train_values = all_sizes_np[training_indices]
X_test_conv = all_components_np[test_indices]
X_test_values = all_sizes_np[test_indices]

y_train = onehot_encoded[training_indices]
y_test = onehot_encoded[test_indices]



In [ ]:


import tensorflow as tf



In [ ]:


import tensorflow.keras
from tensorflow.keras.layers import Flatten, Input, concatenate, Dense, Activation, Dropout, BatchNormalization,  MaxPooling2D, AveragePooling2D, Conv2D
from tensorflow.keras.models import Model
from tensorflow.keras.applications import VGG19
from tensorflow.keras.preprocessing.image import ImageDataGenerator



In [ ]:


# CNN - VGG
X_conv = Input(shape=(64, 64, 3))

vgg_model = VGG19(include_top=False, weights='imagenet')(X_conv)    # Add all the layers of the VGG19 model
# vgg_model.trainable = False
# vgg_model[-1].trainable = True
## Eventueel naar voor terugschuiven om op false te zetten

x_1 = Flatten(name='flatten')(vgg_model)
x_1 = Dense(512, activation='relu', name='fully-connected-1')(x_1)
x_1 = Dense(128, activation='relu', name='fully-connected-2')(x_1)
x_1 = Dense(16, activation='relu', name='fully-connected-3')(x_1)

# x_1 = Dense(6, activation='relu', name='fully-connected-TEST')(x_1)
# final_model = Model(inputs=X_conv, outputs=x_1)

X_extra = Input(shape=(1,))

combined = concatenate([x_1, X_extra, X_extra])
x_2 = Dense(8, activation='relu', name='combined-fully-connected-1')(combined)
## Output
x_2 = Dense(len(components_in_pack), activation='softmax', name='combined-fully-connected-2')(x_2)

final_model = Model(inputs=[X_conv, X_extra, X_extra], outputs=x_2)

opt = tensorflow.keras.optimizers.Adam(learning_rate=1e-4, clipnorm=1.0, clipvalue=0.6)

final_model.compile(optimizer=opt, loss='categorical_crossentropy', 
                   metrics=['accuracy'])


# Generate more data!


In [ ]:


train_generator = ImageDataGenerator(rotation_range=270)



In [ ]:


time_to_repeat_generator = 5
vgg_generator = train_generator.flow([np.repeat(X_train_conv, time_to_repeat_generator, 0), np.repeat(X_train_values, time_to_repeat_generator, 0)], np.repeat(y_train, time_to_repeat_generator, 0), batch_size = 32)



In [ ]:


# test_dat, test_label = vgg_generator[0]


# # In[227]:


# img_nmbr = np.random.randint(len(test_dat[0]))
# plt.imshow(test_dat[0][img_nmbr])
# plt.title(components_in_pack[np.argmax(test_label[img_nmbr])] + ' | diagonal size: ' + str(test_dat[1][img_nmbr]))
# # plt.title(str(test_dat[1][img_nmbr]) + test_label[img_nmbr])



In [ ]:


early_stopping_callback = tensorflow.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10)
reduce_lr = tensorflow.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=20, verbose=1)



In [ ]:


augmented_history = final_model.fit(vgg_generator,
                                    validation_data=([X_test_conv, X_test_values], y_test),
                                    epochs = 30,
                                    steps_per_epoch = 8, # x * batch_size == amount of data in one epoch
                                    verbose = 1,
                                    shuffle=True,
                                    workers=1,
                                    callbacks=[early_stopping_callback, reduce_lr]
                                    )



In [ ]:


# fig=plt.figure(figsize=(16, 6))

# fig.add_subplot(1, 2, 1)
# plt.plot(augmented_history.history['accuracy'])
# plt.plot(augmented_history.history['val_accuracy'])
# plt.title('model accuracy')
# plt.ylabel('accuracy')
# plt.xlabel('epoch')
# plt.legend(['train', 'val'], loc='upper left')

# fig.add_subplot(1, 2, 2)
# plt.plot(augmented_history.history['loss'])
# plt.plot(augmented_history.history['val_loss'])
# plt.title('model loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['train', 'val'], loc='upper left')
# plt.show()



In [ ]:


from sklearn.metrics import classification_report, confusion_matrix, accuracy_score



In [ ]:


predictions = final_model.predict([X_test_conv, X_test_values])
predictions = predictions.argmax(axis=1)

print(f"Accuracy score {accuracy_score(y_test.argmax(axis=1), predictions) * 100}")
print(classification_report(y_test.argmax(axis=1), predictions))

cf = confusion_matrix(y_test.argmax(axis=1), predictions)
print(cf)
# plt.imshow(cf,cmap='RdYlGn_r')



In [ ]:

from uuid import uuid4
final_model.save(f"./models/{pack_name}--{uuid4()}.h5")

